In [11]:
from tkinter import *
from tkcalendar import Calendar,DateEntry  
from tkinter import messagebox
from playsound import playsound    
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import speech_recognition as sr
import pyttsx3
import mysql.connector
from datetime import date
from ttkthemes import ThemedTk
import time
import pyttsx3
from datetime import date
from datetime import datetime
from datetime import timedelta
from twilio.rest import Client 
import pandas
pos_df = pandas.read_excel('TwitterPositive.xlsx', sheet_name='Sheet1')
neu_df = pandas.read_excel('TwitterNeutral.xlsx', sheet_name='Sheet1')
neg_df = pandas.read_excel('TwitterNegative.xlsx', sheet_name='Sheet1')
    
class PlaceholderEntry(ttk.Entry):
    def __init__(self, container, placeholder, *args, **kwargs):
        super().__init__(container, *args, style="Placeholder.TEntry", **kwargs)
        self.placeholder = placeholder
        self.insert("0", self.placeholder)
        self.bind("<FocusIn>", self._clear_placeholder)
        self.bind("<FocusOut>", self._add_placeholder)

    def _clear_placeholder(self, e):
        if self["style"] == "Placeholder.TEntry":
            self.delete("0", "end")
            self["style"] = "TEntry"

    def _add_placeholder(self, e):
        if not self.get():
            self.insert("0", self.placeholder)
            self["style"] = "Placeholder.TEntry"

    
class Application(Frame):
    
    def __init__(self, master=None):
        values=[]
        def analyze(sentence,sess):
            global analyzed_values
            engine = pyttsx3.init()
            engine.setProperty('rate', 150)
            engine.setProperty('volume',1.0)
            voices = engine.getProperty('voices')
            #en_voice_id = "HKEY_LOCAL_MACHINE\SOFTWARE\Microsoft\Speech\Voices\Tokens\TTS_MS_EN-US_ZIRA_11.0"
            engine.setProperty('voice',voices[1].id)
            analyser = SentimentIntensityAnalyzer() 
            snt = analyser.polarity_scores(sentence)
            if sess!=True:
                res_st.configure(text=sentence)
                pos_score.configure(text='Positive : '+str(round(snt['pos']*100,2))+'%',fg="green")
                neu_score.configure(text='Neutral : '+str(round(snt['neu']*100,2))+'%',fg="#f0932b")
                neg_score.configure(text='Negative : '+str(round(snt['neg']*100,2))+'%',fg="red")
                if snt['compound'] >= 0.05 : 
                    overall_lbl.configure(text="Overall : Positive",fg="green") 
                    overall='Positive'
                elif snt['compound'] <= - 0.05 : 
                    overall_lbl.configure(text="Overall : Negative",fg="red") 
                    overall='Negative'

                else : 
                    overall_lbl.configure(text="Overall : Neutral",fg="#f0932b") 
                    overall='Neutral'

                language = 'en'
                mytext='Your Statement was :'+str(round(snt['pos']*100,2))+'% positive,'+str(round(snt['neu']*100,2))+'% neutral,'+str(round(snt['neg']*100,2))+'% negative'
                engine.say(mytext)
                engine.runAndWait()
                print(snt)
            else:
                res_stS.configure(text=sentence)
                pos_scoreS.configure(text='Positive : '+str(round(snt['pos']*100,2))+'%',fg="green")
                neu_scoreS.configure(text='Neutral : '+str(round(snt['neu']*100,2))+'%',fg="#f0932b")
                neg_scoreS.configure(text='Negative : '+str(round(snt['neg']*100,2))+'%',fg="red")
                if snt['compound'] >= 0.05 : 
                    overall_lblS.configure(text="Overall : Positive",fg="green") 
                    overall='Positive'
                elif snt['compound'] <= - 0.05 : 
                    overall_lblS.configure(text="Overall : Negative",fg="red") 
                    overall='Negative'
                else : 
                    overall_lblS.configure(text="Overall : Neutral",fg="#f0932b") 
                    overall='Neutral'
                language = 'en'
                mytext='Your Statement was :'+str(round(snt['pos']*100,2))+'% positive,'+str(round(snt['neu']*100,2))+'% neutral,'+str(round(snt['neg']*100,2))+'% negative'
                engine.say(mytext)
                engine.runAndWait()
            analyzed_values=list((sentence,str(round(snt['pos']*100,2)),str(round(snt['neu']*100,2)),str(round(snt['neg']*100,2)),overall))
            print(analyzed_values[0]+" ------------- pos: "+analyzed_values[1]+" , neu: "+analyzed_values[2]+" , neg: "+analyzed_values[3])
        def insert_values():
            con=mysql.connector.connect(host="localhost",user="project",passwd="mysql123",db="analyzer")
            cur=con.cursor()
            today = date.today()
            today=today.strftime("%d/%m/%Y")
            insert_query="insert into personal_data(name,sex,dob,age,contact,addr,g_name,g_relation,g_contact,doj) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            val=((var_name.get()).title(),var_sex.get(),var_dob.get(),var_age.get(),var_ph.get(),(var_addr.get()).title(),(var_gname.get()).title(),var_relation.get(),var_gph.get(),str(today))
            try:
                dbs=cur.execute(insert_query,val)
                con.commit()
                assign_id()
                raise_frame(TopLeft2)
                
            except Exception as ex:
                print (ex)
                con.rollback()
            print(cur.rowcount,"record inserted!")
            show_updated()
            con.close
        def insert_session():
            print(personid)
            con=mysql.connector.connect(host="localhost",user="project",passwd="mysql123",db="analyzer")
            cur=con.cursor()
            global sess_id
            insert_query="insert into session_table(PersonID) values ({})".format(personid)
            try:
                dbs=cur.execute(insert_query)
                cur.execute("select * from session_table ORDER BY SessionID DESC LIMIT 1")
                result=cur.fetchall() 
                sess_id=list(result[0])
                print(sess_id)
                sessionid_label.place(x=400,y=20,anchor=CENTER)
                str_sessionid_label="Session ID : "+str(sess_id[0])
                #pos=str_phone.find(':')
                sessionid_label.delete("1.0",END)
                sessionid_label.insert(INSERT,str_sessionid_label ) 
                sessionid_label.tag_add("label_font", "1.0",END)
                sessionid_label.tag_config("label_font",font=("Times",18))
                con.commit()
                #assign_id()
                #raise_frame(TopLeft2)
                
            except Exception as ex:
                print (ex)
                con.rollback()
            print(cur.rowcount,"record inserted!")
            raise_frame(Bottom_Session)
            #show_updated()
            con.close
        def insert_statement():
            con=mysql.connector.connect(host="localhost",user="project",passwd="mysql123",db="analyzer")
            cur=con.cursor()
            
            try:
                
                insert_st_query="insert into statement_table (SessionID,statements,pos,neu,neg,overall) values (%s,%s,%s,%s,%s,%s)"
                val=(sess_id[0],analyzed_values[0],analyzed_values[1],analyzed_values[2],analyzed_values[3],analyzed_values[4])
                dbs=cur.execute(insert_st_query,val)
                con.commit()
                saved.pack(side=BOTTOM,anchor=SW)
                #assign_id()
                #raise_frame(TopLeft2)
                
            except Exception as ex:
                print (ex)
                con.rollback()
            print(cur.rowcount,"record inserted!")
            
            
            #raise_frame(Bottom_Session)
            #show_updated()
            con.close
        def previous_session():
            global global_results
            send_btn['state']=DISABLED
            for row in prev_tree.get_children():
                    prev_tree.delete(row)  
            con=mysql.connector.connect(host="localhost",user="project",passwd="mysql123",db="analyzer")
            cur=con.cursor()
            try:
                all_sessions_results=[]
                overall_session_dict={'Positive':0,'Neutral':0,'Negative':0}
                cur.execute("select * from session_table ses,personal_data per where ses.PersonID=per.id AND per.id="+str(personid))
                global_results=cur.fetchall()
                if  global_results!=None and len(global_results)>0:
                    for val in global_results:
                        prev_tree.insert('', 'end', values = (val[1], val[0], val[6],val[2],val[3], val[4]),tag='tree_font')
                        all_sessions_results.append(val[4])
                    for item in all_sessions_results:
                        if item=='Positive':
                            overall_session_dict['Positive']+=1
                        elif item=='Neutral':
                            overall_session_dict['Neutral']+=1
                        else:
                            overall_session_dict['Negative']+=1
                    overall_conclusion=max(overall_session_dict,key=overall_session_dict.get)
                    if overall_conclusion=='Positive':
                        status.place(x=10,y=390)
                        str_status="Status :  GOOD"
                        status.delete("1.0",END )
                        status.insert(INSERT,str_status ) 
                        status.tag_add("status_font", "1.0",END)
                        status.tag_config("status_font",font=("Times",18),foreground='green')
                    elif overall_conclusion=='Neutral':
                        status.place(x=10,y=390)
                        str_status="Status :  OK"
                        status.delete("1.0",END )
                        status.insert(INSERT,str_status ) 
                        status.tag_add("status_font", "1.0",END)
                        status.tag_config("status_font",font=("Times",18),foreground='yellow')
                    else:
                        status.place(x=10,y=380)
                        str_status="Status :  BAD"
                        status.delete("1.0",END )
                        status.insert(INSERT,str_status ) 
                        status.tag_add("status_font", "1.0",END)
                        status.tag_config("status_font",font=("Times",18),foreground='red')
                        send_btn['state']=NORMAL
                else:
                    generate_error("Notice","No Record Found!\nTry Again...")
                raise_frame(Bottom_prev_Session)
            except Exception as ex:
                print (ex)
                con.rollback()
            con.close
            
        def assign_id(): 
            con=mysql.connector.connect(host="localhost",user="project",passwd="mysql123",db="analyzer")
            cur=con.cursor()
            try:
                cur.execute("select * from personal_data ORDER BY id DESC LIMIT 1")
                result=cur.fetchall()
                result=list(result[0])
                ass_id.insert(END, result[0])
                ass_id.tag_add("data_font", '1.'+str(len(str_ass_id)),END)
                ass_id.tag_config("data_font",font=(" ",18),foreground='#944C4D')
                ass_id.configure(state='disabled')
                
                s_name.insert(END, result[1])
                s_name.tag_add("data_font", '1.'+str(len(str_s_name)),END)
                s_name.tag_config("data_font",font=(" ",18),foreground='#944C4D')
                s_name.configure(state='disabled')
                
                s_doj.insert(END, result[-1])
                s_doj.tag_add("data_font", '1.'+str(len(str_s_doj)),END)
                s_doj.tag_config("data_font",font=(" ",18),foreground='#944C4D')
                s_doj.configure(state='disabled')
                #print(result)
            except Exception as ex:
                print(ex)
                con.rollback()
            con.close
        def get_value_db(filters): 
            con=mysql.connector.connect(host="localhost",user="project",passwd="mysql123",db="analyzer")
            cur=con.cursor()
            try:
                if filters==None:
                    cur.execute("select * from personal_data")
                    result=cur.fetchall()
                    return result
                else:
                    cur.execute("select * from personal_data where id="+filters)
                    result=cur.fetchall()
                    return result
            except:
                con.rollback()
            con.close
        def show_updated():
            result=get_value_db(None)
            if len(result)>0:
                val =result[len(result)-1]
                tree.insert('', 'end', values = (val[0], val[1], val[2],val[3], val[4], val[5],val[6], val[7], val[8],val[9]),tag='tree_font' )
        def show_data(filters):
            pid.delete("1.0",END)
            results=get_value_db(filters)
            for row in tree.get_children():
                    tree.delete(row)
            if  results!=None and len(results)>0:
                for val in results:
                    tree.insert('', 'end', values = (val[0], val[1], val[2],val[3], val[4], val[5],val[6], val[7], val[8],val[9]),tag='tree_font' )
            else:
                generate_error("Notice","No Record Found!\nTry Again...")
        def bar(session): 
            if session:
                progressS['value'] = 10
                Bottom_Session.update_idletasks() 
                time.sleep(1) 
                waitS_label.pack(side=BOTTOM)
                progressS['value'] = 20
                Bottom_Session.update_idletasks() 
                time.sleep(1) 

                progressS['value'] = 50
                Bottom_Session.update_idletasks() 
                time.sleep(1) 

                progressS['value'] = 60
                Bottom_Session.update_idletasks() 
                time.sleep(1) 

                progressS['value'] = 70
                Bottom_Session.update_idletasks() 
                time.sleep(1) 
                waitS_label.pack_forget()
                progressS['value'] = 100

                Bottom_Session.update_idletasks()
                time.sleep(1) 
                analyze(statement_entry.get(),session)
                saved.pack_forget()
                raise_frame(Bottom_results_session)
                progressS.pack_forget()
            else:
                progress['value'] = 10
                Bottom.update_idletasks() 
                time.sleep(1) 
                wait_label.pack(side=BOTTOM)
                progress['value'] = 20
                Bottom.update_idletasks() 
                time.sleep(1) 

                progress['value'] = 50
                Bottom.update_idletasks() 
                time.sleep(1) 

                progress['value'] = 60
                Bottom.update_idletasks() 
                time.sleep(1) 

                progress['value'] = 70
                Bottom.update_idletasks() 
                time.sleep(1) 
                wait_label.pack_forget()
                progress['value'] = 100

                Bottom.update_idletasks()
                time.sleep(1) 
                analyze(stmnt_entry.get(),session)
                raise_frame(Bottom_results)
                progress.pack_forget()
        def send_value(data):
            print(data)
            global personid
            personid=data[0]
            pid.place(x=180,y=80,anchor=CENTER)
            str_id="Person ID        :  "+str(data[0])
            #pos=str_id.find(':')
            pid.insert(INSERT,str_id ) 
            pid.tag_add("label_font", "1.0",END)
            pid.tag_config("label_font",font=("Times",14))
            
            pidS.place(x=60,y=100,anchor=CENTER)
            str_idS="ID :  "+str(data[0])
            #pos=str_id.find(':')
            pidS.delete("1.0",END )
            pidS.insert(INSERT,str_idS ) 
            pidS.tag_add("label_font", "1.0",END)
            pidS.tag_config("label_font",font=("Times",14))
            
            name.place(x=300,y=120,anchor=CENTER)
            str_name="Name              :  "+str(data[1])
            #pos=str_name.find(':')
            name.delete("1.0",END )
            name.insert(INSERT,str_name ) 
            name.tag_add("label_font", "1.0",END)
            name.tag_config("label_font",font=("Times",14))
            
            nameS.place(x=250,y=100,anchor=CENTER)
            str_nameS="Name : "+str(data[1])
            #pos=str_name.find(':')
            nameS.delete("1.0",END )
            nameS.insert(INSERT,str_nameS ) 
            nameS.tag_add("label_font", "1.0",END)
            nameS.tag_config("label_font",font=("Times",14))
            
            sex.place(x=180,y=160,anchor=CENTER)
            str_sex="Sex                  :  "+str(data[2])
            #pos=str_sex.find(':')
            sex.delete("1.0",END)
            sex.insert(INSERT,str_sex ) 
            sex.tag_add("label_font", "1.0",END)
            sex.tag_config("label_font",font=("Times",14))
            
            sexS.place(x=450,y=100,anchor=CENTER)
            str_sexS="Sex : "+str(data[2])
            #pos=str_sex.find(':')
            sexS.delete("1.0",END)
            sexS.insert(INSERT,str_sexS ) 
            sexS.tag_add("label_font", "1.0",END)
            sexS.tag_config("label_font",font=("Times",14))
            
            dob.place(x=180,y=200,anchor=CENTER)
            str_dob="Date of Birth    :  "+str(data[3])
            #pos=str_dob.find(':')
            dob.delete("1.0",END)
            dob.insert(INSERT,str_dob ) 
            dob.tag_add("label_font", "1.0",END)
            dob.tag_config("label_font",font=("Times",14))
            
            age.place(x=180,y=240,anchor=CENTER)
            str_age="Age                 :  "+str(data[4])
            #pos=str_age.find(':')
            age.delete("1.0",END)
            age.insert(INSERT,str_age ) 
            age.tag_add("label_font", "1.0",END)
            age.tag_config("label_font",font=("Times",14))
            
            ageS.place(x=550,y=100,anchor=CENTER)
            str_ageS="Age : "+str(data[4])
            #pos=str_age.find(':')
            ageS.delete("1.0",END)
            ageS.insert(INSERT,str_ageS ) 
            ageS.tag_add("label_font", "1.0",END)
            ageS.tag_config("label_font",font=("Times",14))
            
            
            phone.place(x=200,y=280,anchor=CENTER)
            str_phone="Phone              :  "+str(data[5])
            #pos=str_phone.find(':')
            phone.delete("1.0",END)
            phone.insert(INSERT,str_phone ) 
            phone.tag_add("label_font", "1.0",END)
            phone.tag_config("label_font",font=("Times",14))
            
            phoneS.place(x=700,y=100,anchor=CENTER)
            str_phoneS="Phone : "+str(data[5])
            #pos=str_phone.find(':')
            phoneS.delete("1.0",END)
            phoneS.insert(INSERT,str_phoneS ) 
            phoneS.tag_add("label_font", "1.0",END)
            phoneS.tag_config("label_font",font=("Times",14))
            
            addr.place(x=260,y=320,anchor=CENTER)
            str_addr="Address           :  "+str(data[6])
            #pos=str_addr.find(':')
            addr.delete("1.0",END)
            addr.insert(INSERT,str_addr ) 
            addr.tag_add("label_font", "1.0",END)
            addr.tag_config("label_font",font=("Times",14))
            
            g_name.place(x=600,y=80,anchor=CENTER)
            str_gname="Name              :  "+str(data[7])
            #pos=str_name.find(':')
            g_name.delete("1.0",END)
            g_name.insert(INSERT,str_gname ) 
            g_name.tag_add("label_font", "1.0",END)
            g_name.tag_config("label_font",font=("Times",14))

            relation.place(x=600,y=120,anchor=CENTER)
            str_relation="Relation           :  "+str(data[8])
            #pos=str_relation.find(':')
            relation.delete("1.0",END)
            relation.insert(INSERT,str_relation ) 
            relation.tag_add("label_font", "1.0",END)
            relation.tag_config("label_font",font=("Times",14))
            
            gphone.place(x=620,y=160,anchor=CENTER)
            str_gphone="Phone              :  "+str(data[9])
            #pos=str_gphone.find(':')
            gphone.delete("1.0",END)
            gphone.insert(INSERT,str_gphone ) 
            gphone.tag_add("label_font", "1.0",END)
            gphone.tag_config("label_font",font=("Times",14))
            
            
           
        def selectItem(a):
            curItem = tree.focus()
            item=tree.item(curItem)
            print(item)
            send_value(item['values'])
            raise_frame(Bottom_P_details)
            startS.configure(state='normal')
            prev_record.configure(state='normal')
        def search():
            value=search_inp.get()
            if value!="" or value!="Enter ID / Name":
                show_data(value)
            else:
                generate_error("Error","Invalid Input!")
            operate(Bottom_details,TopRight3)
        def show_all_data():
            show_data(None)
            operate(Bottom_details,TopRight3)
        def clicked():
            if stmnt_entry.get()=="" and statement_entry.get()=="":
                generate_error("Invalid Input","No Statement Given.")
            else:
                switchButtonState(analyze_btn)
                if stmnt_entry.get()=="":
                    progressS.pack(side=BOTTOM)
                    bar(True)
                else:
                    progress.pack(side=BOTTOM)
                    bar(False)
        def switchButtonState(button):
            if (button['state'] == NORMAL):
                button['state'] = DISABLED
            else:
                button['state'] = NORMAL
        def raise_frame(frame):
            frame.tkraise()
        def on_off(o):
            Existing_P.configure(state='disabled')
            
        def cncl_on_off(o):
            Existing_P.configure(state='normal')
            
        def show_on_off(o):
            startS.configure(state='disabled')
            prev_record.configure(state='disabled')
        def operate(frame,widget1=None,widget2=None,widget3=None,widget4=None):
            if isinstance(widget1, Button):
                switchButtonState(widget1)
            if isinstance(widget1, Frame):
                raise_frame(widget1)
            if isinstance(widget2, Entry):
                widget2.delete(0,END)
            if isinstance(widget3, Button):
                switchButtonState(widget3)
            if isinstance(widget4, Button):
                switchButtonState(widget4)
            raise_frame(frame)
        def finish():
            stmnt_entry.delete(0,END)
            statement_entry.delete(0,END)
            raise_frame(Bottom)
        def cont():
            stmnt_entry.delete(0,END)
            statement_entry.delete(0,END)
            raise_frame(Bottom_Session)
            
        def end_session():
            con=mysql.connector.connect(host="localhost",user="project",passwd="mysql123",db="analyzer")
            cur=con.cursor()
            overall_dict={'Positive':0,'Neutral':0,'Negative':0}
            try:
                cur.execute("select * from statement_table where SessionID = "+str(sess_id[0]))
                result=cur.fetchall()
                for x in result:
                    if x[-1]=='Positive':
                        overall_dict['Positive']+=1
                    elif x[-1]=='Neutral':
                        overall_dict['Neutral']+=1
                    else:
                        overall_dict['Negative']+=1

                    print(x)
                print(len(result))
                print(max(overall_dict))
                ratio=str(overall_dict['Positive'])+" : "+str(overall_dict['Neutral'])+" : "+str(overall_dict['Negative'])
                update_query="UPDATE session_table SET total_sts = %s,ratio = %s, conclusion = %s WHERE SessionID = %s"
                val=(len(result),ratio,max(overall_dict,key=overall_dict.get),sess_id[0])
                cur.execute(update_query, val)
                con.commit()
            except Exception as Ex:
                print(Ex)
                con.rollback()
            con.close
            stmnt_entry.delete(0,END)
            statement_entry.delete(0,END)
            raise_frame(Bottom)
            raise_frame(TopRight)
            
        def send_alert():
            def send_sms(phone):
                try:
                    account_sid = 'AC41e31dd9772991d1a68e3840b30e158f' 
                    auth_token = '8564ee69bf659885cc96569e4af58f91' 
                    client = Client(account_sid, auth_token) 
                    message = client.messages.create( 
                            from_='+12513026259', 
                            to=phone,
                            body="\n\nNeed Attention\n\nDetails Mentioned Below -\n"+str_personid+str_name_sms+str_sex_sms+str_age_sms+str_phone
                    )
                except Exception as Ex:
                    print(Ex)
            
            phones=['+917014513360']
            con=mysql.connector.connect(host="localhost",user="project",passwd="mysql123",db="analyzer")
            cur=con.cursor()
            try:
                cur.execute("select * from personal_data where id="+str(personid))
                results=cur.fetchall()
                result=results[0]
                str_personid="Person ID: {}\n".format(result[0])
                str_name_sms="Name: {}\n".format(result[1])
                str_sex_sms="Sex: {}\n".format(result[2])
                str_age_sms="Age : {}\n".format(result[4])
                str_phone="Phone : {}\n".format(result[5])
                for ph in phones:
                    send_sms(ph)
            except Exception as Ex:
                print(Ex)
                con.rollback()
            con.close
        def fill_metrics(category):
            for row in metrics_tree.get_children():
                    metrics_tree.delete(row)
            if category=='pos':
                for each_rec in range(len(pos_df)):
                        metrics_tree.insert("", 'end', values=list(pos_df.loc[each_rec]),tag='tree_font')
                raise_frame(Bottom_metrics_table)
            elif category=='neu':
                for each_rec in range(len(neu_df)):
                        metrics_tree.insert("", 'end', values=list(neu_df.loc[each_rec]),tag='tree_font')
                raise_frame(Bottom_metrics_table)
            else:
                for each_rec in range(len(neg_df)):
                        metrics_tree.insert("", 'end', values=list(neg_df.loc[each_rec]),tag='tree_font')
                raise_frame(Bottom_metrics_table)
        def generate_error(error_title,msg):
            messagebox.showerror(error_title,msg)
        def get_value():
            a=entry_name.get()
            b=var_sex.get()
            c=entry_contact.get()
            d=entry_addr.get()
            e=entry_age.get()
            f=entry_gname.get()
            g=entry_gcontact.get()
            if a=="First Name*" or b=="" or c=="Contact No.*" or d=="City*" or e=="" or f=="Gaurdian Name*" or g=="Gaurdian Contact*":
                generate_error("Invalid Input","Please provide required(*) feilds!")
            else:
                insert_values()
        def validate_num(new_value):
            return new_value.isdigit() or new_value=="Contact No.*" or new_value=="Gaurdian Contact*"
        def validate_alpha(new_value):
            return new_value.isalpha() or new_value.isspace() or new_value=="Full Name*" or new_value=="." or new_value=="Gaurdian Name*"
        def on_name(*args):
            val_name = var_name.get()
            if val_name=="Full Name*" or val_name=="Gaurdian Name*":
                pass
            else:
                if len(val_name) > 20:
                    var_name.set(val_name[:20])
        def on_age(*args):
            val_age = var_age.get()
            if len(val_age) > 2:
                var_age.set(val_age[:2])
        def on_phone(*args):
            val_ph = var_ph.get()
            if len(val_ph) > 12:
                var_ph.set(val_ph[:12])
        def exit_fn():
            root.destroy()
        
        #---------------------------------------M a s t e r  P a g e---------------------------------#
        Frame.__init__(self, master)
        self.grid()
        for r in range(6):
            self.master.rowconfigure(r, weight=1)    
        for c in range(5):
                self.master.columnconfigure(c, weight=1)
        s = ttk.Style()
        s.configure('my.TButton', font=('Helvetica', 18))
        #b = ttk.Button(mainframe, text='Press me', style='my.TButton',command=foo)
        analyze_btn=ttk.Button(master, text="Analyze",takefocus=False,command=lambda: clicked(),style='my.TButton')
        analyze_btn.grid(row=6,column=0,columnspan=4,sticky=W+E+N+S,ipady=15)
        record_btn=ttk.Button(master, text="Record",takefocus=False,style='my.TButton')
        record_btn.grid(row=6,column=4,columnspan=2,sticky=W+E+N+S,ipady=15)

        TopLeft2 = Frame(master,borderwidth=4,relief="ridge",bg='#f1f1b0')
        TopLeft2.grid(row = 0, column = 0, rowspan=3,columnspan = 4, sticky = W+E+N+S) 
        TopLeft = Frame(master,borderwidth=4,relief="ridge",bg='#f1f1b0')
        TopLeft.grid(row = 0, column = 0, rowspan=3,columnspan = 4, sticky = W+E+N+S) 
        TopLeft_intro = Frame(master,borderwidth=4,relief="ridge",bg='#f1f1b0')
        TopLeft_intro.grid(row = 0, column = 0, rowspan=3,columnspan = 4, sticky = W+E+N+S)
        
        
        Bottom_metrics = Frame(master,borderwidth=4,relief="ridge",bg='#f1f1b0')
        Bottom_metrics.grid(row = 3, column = 0, rowspan = 3, columnspan = 5, sticky = W+E+N+S)
        Bottom_metrics_table = Frame(master,borderwidth=4,relief="ridge",bg='#f1f1b0')
        Bottom_metrics_table.grid(row = 3, column = 0, rowspan = 3, columnspan = 5, sticky = W+E+N+S)
        Bottom_P_details = Frame(master,borderwidth=4,relief="ridge",bg='#f1f1b0')
        Bottom_P_details.grid(row = 3, column = 0, rowspan = 3, columnspan = 5, sticky = W+E+N+S)
        Bottom_details = Frame(master,borderwidth=4,relief="ridge",bg='#f1f1b0')
        Bottom_details.grid(row = 3, column = 0, rowspan = 3, columnspan = 5, sticky = W+E+N+S)
        Bottom_results = Frame(master,borderwidth=4,relief="ridge",bg='#f1f1b0')
        Bottom_results.grid(row = 3, column = 0, rowspan = 3, columnspan = 5, sticky = W+E+N+S)
        Bottom_Session = Frame(master,borderwidth=4,relief="ridge",bg='#f1f1b0')
        Bottom_Session.grid(row = 3, column = 0, rowspan = 3, columnspan = 5, sticky = W+E+N+S)
        Bottom_prev_Session = Frame(master,borderwidth=4,relief="ridge",bg='#f1f1b0')
        Bottom_prev_Session.grid(row = 3, column = 0, rowspan = 3, columnspan = 5, sticky = W+E+N+S)
        Bottom_results_session = Frame(master,borderwidth=4,relief="ridge",bg='#f1f1b0')
        Bottom_results_session.grid(row = 3, column = 0, rowspan = 3, columnspan = 5, sticky = W+E+N+S)
        Bottom = Frame(master,borderwidth=4,relief="ridge",bg='#f1f1b0')
        Bottom.grid(row = 3, column = 0, rowspan = 3, columnspan = 5, sticky = W+E+N+S)
        
        
        
        
        TopRight2 = Frame(master,borderwidth=4,relief="ridge",bg='#f1f1b0')
        TopRight2.grid(row = 0, column = 4, rowspan = 3, sticky = W+E+N+S)
        TopRight3 = Frame(master,borderwidth=4,relief="ridge",bg='#f1f1b0')
        TopRight3.grid(row = 0, column = 4, rowspan = 3, sticky = W+E+N+S)
        TopRight = Frame(master,borderwidth=4,relief="ridge",bg='#f1f1b0')
        TopRight.grid(row = 0, column = 4, rowspan = 3, sticky = W+E+N+S)       
        
        #---------------------------------------------------------------------------------------------#
        
        #--------------------------T O P - L E F T  P A N N E L--------------------------------------#
           
        Title_intro=Label(TopLeft_intro,text="Versatile Sentiment Analyzer",font=("Times",30),bg='#f1f1b0',fg="#0c2461")
        Title_intro.place(x=70,y=30)
        metrics_btn=ttk.Button(TopLeft_intro, text="Model Metrics",takefocus=False,command=lambda: raise_frame(Bottom_metrics))
        metrics_btn.pack(side=BOTTOM,anchor=SE)
        
        check_num = TopLeft.register(validate_num)
        check_alpha = TopLeft.register(validate_alpha)
        
        Title=Label(TopLeft,text="Versatile Sentiment Analyzer",font=("",12),fg='#2c3e50')
        Title.pack()
        label_frame = LabelFrame(TopLeft, text = 'Enter Your Details',borderwidth=4,relief='ridge') 
        label_frame.pack(expand='yes',fill='both')
        
        var_name = StringVar();var_ph = StringVar();var_addr = StringVar();var_dob = StringVar();var_age = StringVar();var_gname=StringVar();var_gph=StringVar();var_gaddr=StringVar()
        var_name.trace_variable("w", on_name)
        entry_name = PlaceholderEntry(TopLeft, "Full Name*",font=("",12),validate="key",validatecommand=(check_alpha, "%S"),textvariable=var_name,justify='center')
        entry_name.place(x=30,y=47,width=380,height=30)
        
        L_sex = Label(TopLeft,text="Sex* : ",font=("",12))
        L_sex.place(x=90,y=80)
        var_sex = StringVar()
        R_m = Radiobutton(TopLeft, text="Male", variable=var_sex, value="Male")
        R_m.place(x=22,y=105)
        R_f = Radiobutton(TopLeft, text="Female", variable=var_sex, value="Female")
        R_f.place(x=72,y=105)
        R_o = Radiobutton(TopLeft, text="Other", variable=var_sex, value="Others")
        R_o.place(x=135,y=105)
        
        var_ph.trace_variable("w", on_phone)
        entry_contact = PlaceholderEntry(TopLeft, "Contact No.*",font=("",12),validate="key",validatecommand=(check_num, "%S"),textvariable=var_ph,justify='center')
        entry_contact.place(x=30,y=137,width=180,height=30)
        
        entry_addr = PlaceholderEntry(TopLeft, "Address*",font=("",12),textvariable=var_addr,justify='center')
        entry_addr.place(x=30,y=180,width=380,height=30)     

        L_dob = Label(TopLeft,text="DOB* : ",font=("",12))
        L_dob.place(x=230,y=95)
        entry_dob = DateEntry(TopLeft, width=12,text="Date", background='darkblue',foreground='white', borderwidth=2,textvariable=var_dob)
        entry_dob.place(x=300,y=95,width=105,height=30)
        
        L_sex = Label(TopLeft,text="Age* :   ",font=("",12))
        L_sex.place(x=230,y=137)
        
        var_age.trace_variable("w", on_age)
        entry_age = PlaceholderEntry(TopLeft,"",validate="all", validatecommand=(check_num, "%S"),textvariable=var_age,justify='center')
        entry_age.place(x=330,y=137,width=60,height=30)
        
        var_gname.trace_variable("w", on_name)
        entry_gname = PlaceholderEntry(TopLeft, "Gaurdian Name*",font=("",12),validate="key",validatecommand=(check_alpha, "%S"),textvariable=var_gname,justify='center')
        entry_gname.place(x=430,y=47,width=180,height=30)
        
        OPTIONS = ["Relation","Father","Mother","Brother","Sister","Uncle","Aunt"] #etc
        var_relation = StringVar(TopLeft)
        var_relation.set(OPTIONS[0]) # default value
        relation_dd = ttk.OptionMenu(TopLeft, var_relation, *OPTIONS)
        relation_dd.place(x=430,y=95,width=180,height=30)

        entry_gcontact = PlaceholderEntry(TopLeft, "Gaurdian Contact*",validate="key",validatecommand=(check_num, "%S"),font=("",12),justify='center',textvariable=var_gph)
        entry_gcontact.place(x=430,y=137,width=180,height=30)
        
        entry_gaddr = PlaceholderEntry(TopLeft, "Gaurdian Addr.",font=("",12),justify='center')
        entry_gaddr.place(x=430,y=180,width=180,height=30)
        
        cncl_btn=ttk.Button(TopLeft, text="Cancel",takefocus=False,command=lambda: operate(TopLeft_intro,analyze_btn))
        cncl_btn.pack(side=RIGHT)
        cncl_btn.bind("<Button-1>",cncl_on_off)
        cnf_btn=ttk.Button(TopLeft, text="Confirm",takefocus=False,command=lambda: get_value())
        cnf_btn.pack()
        
        ass_id= Text(TopLeft2,height=2,width=60,bg='#f1f1b0',relief='flat') 
        ass_id.place(x=300,y=120,anchor=CENTER)
        str_ass_id="Assigned ID                 :  "
        pos=str_ass_id.find(':')
        ass_id.insert(INSERT,str_ass_id ) 
        ass_id.tag_add("label_font", "1.0",'1.'+str(pos))
        ass_id.tag_config("label_font",font=("Times",12))
        
        s_name= Text(TopLeft2,height=2,width=60,bg='#f1f1b0',relief='flat') 
        s_name.place(x=300,y=160,anchor=CENTER)
        str_s_name="Name                          :  "
        pos=str_s_name.find(':')
        s_name.insert(INSERT,str_s_name ) 
        s_name.tag_add("label_font", "1.0",'1.'+str(pos))
        s_name.tag_config("label_font",font=("Times",12))
        
        s_doj= Text(TopLeft2,height=2,width=60,bg='#f1f1b0',relief='flat') 
        s_doj.place(x=300,y=200,anchor=CENTER)
        str_s_doj="Membership Date        :  "
        pos=str_s_doj.find(':')
        s_doj.insert(INSERT,str_s_doj ) 
        s_doj.tag_add("label_font", "1.0",'1.'+str(pos))
        s_doj.tag_config("label_font",font=("Times",12))
        
        success_label= Text(TopLeft2,height=3,width=50,bg='#f1f1b0',relief='flat') 
        success_label.place(x=370,y=50,anchor=CENTER)
        str_success="Details Saved Successfully!"
        success_label.insert(INSERT,str_success ) 
        success_label.tag_add("label_font", "1.0",END)
        success_label.tag_config("label_font",font=("Times",24),foreground='#27ae60')
        
        go_bck_btn=ttk.Button(TopLeft2, text="Go Back",takefocus=False,command=lambda: raise_frame(TopLeft_intro) )
        go_bck_btn.pack(side=TOP, anchor=NW)
        startS=ttk.Button(TopLeft2, text="Start Session",width=25,takefocus=False,command=lambda: raise_frame(Bottom_Session))
        startS.pack(side=BOTTOM, anchor=SE)
        #
        
        #-------------------------------------------------------------------------------------------#
        
        #--------------------------T O P - R I G H T  P A N N E L------------------------------------#
        
        lbl1=Label(TopRight,text="Options",font=("",8),bg='#f1f1b0')
        lbl1.place(x=2,y=5)
        
        New_P=ttk.Button(TopRight, text="New Person",width=20,takefocus=False,command=lambda: operate(TopLeft,analyze_btn,stmnt_entry))
        New_P.place(x=12,y=50)
        #New_P.bind("<Button-1>",on_off)
        
        Existing_P=ttk.Button(TopRight, text="Existing Person",width=20,takefocus=False,command=lambda: raise_frame(TopRight2))
        Existing_P.place(x=12,y=100)
        
        show_data_btn=ttk.Button(TopRight, text="Show All Data",width=20,takefocus=False,command=lambda: show_all_data())
        show_data_btn.place(x=12,y=150)
        show_data_btn.bind("<Button-1>",show_on_off)
        
        
        exit_btn=ttk.Button(TopRight, text="Exit",width=20,takefocus=False,command=lambda: exit_fn())
        exit_btn.place(x=12,y=200)
        
        lbl1=Label(TopRight2,text="Options",font=("",8),bg='#f1f1b0')
        lbl1.place(x=2,y=5)
        
        search_inp = PlaceholderEntry(TopRight2, "Enter ID / Name",font=("",10))
        search_inp.place(x=12,y=50,width=130,height=30)
        
        search_btn=ttk.Button(TopRight2, text="Search",width=20,takefocus=False,command=lambda: search())
        search_btn.place(x=12,y=100)
        
        show_data_btn=ttk.Button(TopRight2, text="Show All Data",width=20,takefocus=False,command=lambda: operate(Bottom_details,TopRight3))
        show_data_btn.place(x=12,y=150)
        show_data_btn.bind("<Button-1>",show_on_off)
        
        back2_btn=ttk.Button(TopRight2, text="Back",width=20,takefocus=False,command=lambda: operate(Bottom,TopRight))
        back2_btn.place(x=12,y=200)
        
        lbl2=Label(TopRight3,text="Options",font=("",8),bg='#f1f1b0')
        lbl2.place(x=2,y=5)
        
        startS=ttk.Button(TopRight3, text="Start Session",width=20,takefocus=False,command=lambda: insert_session())
        startS.place(x=12,y=50)
        
        prev_record=ttk.Button(TopRight3, text="Previous Session",width=20,takefocus=False,command=lambda: previous_session())
        prev_record.place(x=12,y=100)
        
        back_btn=ttk.Button(TopRight3, text="Back",width=20,takefocus=False,command=lambda: operate(TopRight,Bottom))
        back_btn.place(x=12,y=150)
        
        exit_btn=ttk.Button(TopRight3, text="Exit",width=20,takefocus=False,command=lambda: exit_fn())
        exit_btn.place(x=12,y=200)
        #-------------------------------------------------------------------------------------------#
        
        #-------------------------------B O T T O M -  P A N N E L----------------------------------#
                                #********************************************************#
        
        metrics_title=Label(Bottom_results_session,text="Model Metrics",font=("Times",18,"underline"),bg='#f1f1b0')
        metrics_title.place(x=400,y=30,anchor=CENTER)

        dataset_lbl=Label(Bottom_results_session,text="Dataset Used : ",font=("",12),bg='#f1f1b0')
        dataset_lbl.place(x=100,y=80,anchor=CENTER)
        
        used_st=Label(Bottom_results_session,text="Twitter Comments Dataset",font=("Times",22,"bold"),bg='#f1f1b0')
        used_st.place(x=400,y=130,anchor=CENTER)
        
        which_has=Label(Bottom_results_session,text="which has",font=("",10),bg='#f1f1b0')
        which_has.place(x=400,y=200,anchor=CENTER)
        
        no_pos=Label(Bottom_results_session,text="",font=("",16),bg='#f1f1b0')
        no_pos.place(x=130,y=250,anchor=CENTER)
        
        neu_scoreS=Label(Bottom_results_session,text="",font=("",16),bg='#f1f1b0')
        neu_scoreS.place(x=400,y=250,anchor=CENTER)
        
        neg_scoreS=Label(Bottom_results_session,text="",font=("",16),bg='#f1f1b0')
        neg_scoreS.place(x=680,y=250,anchor=CENTER)
        
        overall_lblS=Label(Bottom_results_session,text="",font=("",20),bg='#f1f1b0')
        overall_lblS.place(x=400,y=330,anchor=CENTER)
        
        saved= Text(Bottom_results_session,height=2,width=30,fg='GREEN',bg='#f1f1b0',relief='flat')
        saved.insert(INSERT,"                   Saved!")
        saved.tag_add("label_font", "1.0",END)
        saved.tag_config("label_font",font=("Times",20))
        
        s1 = ttk.Style()
        s1.configure('smallBtn.TButton', font=('Helvetica', 14))
        save_btn=ttk.Button(Bottom_results_session, text="Save",width=10,takefocus=False,style='smallBtn.TButton',command=lambda: insert_statement())
        save_btn.place(x=10,y=390)
        continue_btn=ttk.Button(Bottom_results_session, text="Continue",width=10,takefocus=False,style='smallBtn.TButton',command=lambda: cont())
        continue_btn.place(x=350,y=390)
        endS_btn=ttk.Button(Bottom_results_session, text="End Session",width=10,takefocus=False,style='smallBtn.TButton',command=lambda: end_session())
        endS_btn.place(x=665,y=390)
        
                                #********************************************************#
        
        try_lbl=Label(Bottom,text="Try It Now !",font=("Times",20,"italic"),fg='#718093',bg='#f1f1b0')
        try_lbl.place(x=400,y=30,anchor=CENTER)
        inp_lbl=Label(Bottom,text="Give Your Statement :",font=("",12),bg='#f1f1b0')
        inp_lbl.place(x=120,y=100,anchor=CENTER)

        stmnt_entry=Entry(Bottom,width=40,font=("",20),fg='#944C4D',bg='White')
        stmnt_entry.place(x=400,y=150,anchor=CENTER)
        stmnt_entry.bind("<Return>",(lambda event: clicked()))
        stmnt_entry.focus_force()

        wait_label=Label(Bottom,text="Please Wait! Your sentence is being processed...",font=("Helvetica",10),bg='#f1f1b0')
        progress = ttk.Progressbar(Bottom, orient = HORIZONTAL, length = 800, mode = 'determinate')
                                #********************************************************#
            
                                #********************************************************#
        result_title=Label(Bottom_results,text="Your Results are Ready!",font=("Times",18,"underline"),bg='#f1f1b0')
        result_title.place(x=400,y=30,anchor=CENTER)

        stmnt_lbl=Label(Bottom_results,text="Your Statemant was : ",font=("",12),bg='#f1f1b0')
        stmnt_lbl.place(x=100,y=80,anchor=CENTER)
        
        res_st=Label(Bottom_results,text="",font=("Times",22,"bold"),bg='#f1f1b0')
        res_st.place(x=400,y=130,anchor=CENTER)
        
        which_is=Label(Bottom_results,text="which is",font=("",10),bg='#f1f1b0')
        which_is.place(x=400,y=200,anchor=CENTER)
        
        pos_score=Label(Bottom_results,text="",font=("",16),bg='#f1f1b0')
        pos_score.place(x=130,y=250,anchor=CENTER)
        
        neu_score=Label(Bottom_results,text="",font=("",16),bg='#f1f1b0')
        neu_score.place(x=400,y=250,anchor=CENTER)
        
        neg_score=Label(Bottom_results,text="",font=("",16),bg='#f1f1b0')
        neg_score.place(x=680,y=250,anchor=CENTER)
        
        overall_lbl=Label(Bottom_results,text="",font=("",20),bg='#f1f1b0')
        overall_lbl.place(x=400,y=330,anchor=CENTER)
        
        s1 = ttk.Style()
        s1.configure('smallBtn.TButton', font=('Helvetica', 14))
        finish_btn=ttk.Button(Bottom_results, text="Finish",width=10,takefocus=False,style='smallBtn.TButton',command=lambda: finish())
        finish_btn.place(x=350,y=370)
        
        
        
                        #********************************************************#
        metrics_title=Label(Bottom_metrics,text="Model Metrics",font=("Times",18,"underline"),bg='#f1f1b0')
        metrics_title.place(x=400,y=30,anchor=CENTER)

        dataset_lbl=Label(Bottom_metrics,text="Dataset Used : ",font=("",12),bg='#f1f1b0')
        dataset_lbl.place(x=100,y=80,anchor=CENTER)
        
        used=Label(Bottom_metrics,text="Twitter Dataset",font=("Times",22,"bold"),bg='#f1f1b0',fg="#273c75")
        used.place(x=400,y=80,anchor=CENTER)
        
        total_number=Label(Bottom_metrics,text="Dataset Length : 3,707 ",font=("",16),bg='#f1f1b0')
        total_number.place(x=400,y=135,anchor=CENTER)
        
        total_pos=Label(Bottom_metrics,text="Positive Labeled : 1,098",font=("",14),bg='#f1f1b0')
        total_pos.place(x=120,y=200,anchor=CENTER)
        
        total_neu=Label(Bottom_metrics,text="Neutral Labeled : 1,496",font=("",14),bg='#f1f1b0')
        total_neu.place(x=400,y=200,anchor=CENTER)
        
        total_neg=Label(Bottom_metrics,text="Negative Labeled : 1,113",font=("",14),bg='#f1f1b0')
        total_neg.place(x=660,y=200,anchor=CENTER)
        
        outcomes=Label(Bottom_metrics,text="Outcomes",font=("Times",18,),bg='#f1f1b0')
        outcomes.place(x=400,y=250,anchor=CENTER)
        
        predicted_pos=Label(Bottom_metrics,text="Positive Labeled : 968",font=("",14,"bold"),bg='#f1f1b0',fg="#273c75")
        predicted_pos.place(x=120,y=300,anchor=CENTER)
        
        predicted_neu=Label(Bottom_metrics,text="Neutral Labeled : 948",font=("",14,"bold"),bg='#f1f1b0',fg="#273c75")
        predicted_neu.place(x=400,y=300,anchor=CENTER)
        
        predictive_neg=Label(Bottom_metrics,text="Negative Labeled : 716",font=("",14,"bold"),bg='#f1f1b0',fg="#273c75")
        predictive_neg.place(x=660,y=300,anchor=CENTER)
        
        accuracy=Label(Bottom_metrics,text="Accuracy : 71.00%",font=("Times",20,"bold"),bg='#f1f1b0',fg="#20bf6b")
        accuracy.place(x=400,y=350,anchor=CENTER)
        
        sbtn = ttk.Style()
        sbtn.configure('smallBtn.TButton', font=('Helvetica', 14))
        positve_data_btn=ttk.Button(Bottom_metrics, text="Positive Classifications",width=20,takefocus=False,style='smallBtn.TButton',command=lambda:fill_metrics('pos'))
        positve_data_btn.place(x=10,y=390)
        neutral_data_btn=ttk.Button(Bottom_metrics, text="Neutral Classifications",width=20,takefocus=False,style='smallBtn.TButton',command=lambda:fill_metrics('neu'))
        neutral_data_btn.place(x=280,y=390)
        negative_data_btn=ttk.Button(Bottom_metrics, text="Negative Classifications",width=20,takefocus=False,style='smallBtn.TButton',command=lambda:fill_metrics('neg'))
        negative_data_btn.place(x=550,y=390)
        
        metrics_tree = ttk.Treeview(Bottom_metrics_table, columns = (1,2,3), show = "headings")
        metrics_tree.pack(side='left',fill='both',expand=1)
        metrics_tree.tag_configure('tree_font', font="Helvetica 10")
        metrics_style = ttk.Style()
        metrics_style.configure("Treeview.Heading", font=(None, 10))
        metrics_tree.heading(1, text="Labels")
        metrics_tree.heading(2, text="Text")
        metrics_tree.heading(3, text="Classifications")
        metrics_tree.column(1,width = 1,anchor="c")
        metrics_tree.column(2, width = 500,anchor="c")
        metrics_tree.column(3,width = 1,anchor="c")
        scroll = ttk.Scrollbar(Bottom_metrics_table, orient="vertical", command=metrics_tree.yview)
        scroll.pack(side = 'right', fill = 'y')
        
        '''
        saved= Text(Bottom_results_session,height=2,width=30,fg='GREEN',bg='#f1f1b0',relief='flat')
        saved.insert(INSERT,"                   Saved!")
        saved.tag_add("label_font", "1.0",END)
        saved.tag_config("label_font",font=("Times",20))
        
        s1 = ttk.Style()
        s1.configure('smallBtn.TButton', font=('Helvetica', 14))
        save_btn=ttk.Button(Bottom_results_session, text="Save",width=10,takefocus=False,style='smallBtn.TButton',command=lambda: insert_statement())
        save_btn.place(x=10,y=390)
        continue_btn=ttk.Button(Bottom_results_session, text="Continue",width=10,takefocus=False,style='smallBtn.TButton',command=lambda: cont())
        continue_btn.place(x=350,y=390)
        endS_btn=ttk.Button(Bottom_results_session, text="End Session",width=10,takefocus=False,style='smallBtn.TButton',command=lambda: end_session())
        endS_btn.place(x=665,y=390)
        '''
                        #********************************************************#
            
        tree = ttk.Treeview(Bottom_details, columns = (1,2,3,4,5,6,7,8,9,10), show = "headings")
        tree.pack(side='left',fill='both',expand=1)
        tree.tag_configure('tree_font', font="Helvetica 10")
        style = ttk.Style()
        style.configure("Treeview.Heading", font=(None, 10))
        tree.heading(1, text="ID")
        tree.heading(2, text="Name")
        tree.heading(3, text="Sex")
        tree.heading(4, text="DOB")
        tree.heading(5, text="Age")
        tree.heading(6, text="Phone")
        tree.heading(7, text="Address")
        tree.heading(8, text="G_Name")
        tree.heading(9, text="G_Relation")
        tree.heading(10, text="G_Phone")
        tree.column(1, width = 10,anchor="c")
        tree.column(2, width = 80,anchor="c")
        tree.column(3, width = 40,anchor="c")
        tree.column(4, width = 70,anchor="c")
        tree.column(5, width = 30,anchor="c")
        tree.column(6, width = 80,anchor="c")
        tree.column(7, width = 100,anchor="c")
        tree.column(8, width = 80,anchor="c")
        tree.column(9, width = 60,anchor="c")
        tree.column(10, width = 80,anchor="c")

        scroll = ttk.Scrollbar(Bottom_details, orient="vertical", command=tree.yview)
        scroll.pack(side = 'right', fill = 'y')
        
        tree.bind('<Return>', selectItem)
        tree.bind('<Double-1>', selectItem)
        tree.configure(yscrollcommand=scroll.set)
        #show_data_btn.bind("<Button-1>",show_data(None))
        #search_btn.bind("<Button-1>",show_data(search_inp.get()))
        
        prev_tree = ttk.Treeview(Bottom_prev_Session, columns = (1,2,3,4,5,6), show = "headings")
        prev_tree.pack(side=LEFT,anchor=N,fill='both',expand=1)
        prev_tree.tag_configure('tree_font', font="Helvetica 10")
        style = ttk.Style()
        style.configure("Treeview.Heading", font=(None, 10))
        prev_tree.heading(1, text="PersonID")
        prev_tree.heading(2, text="SessionID")
        prev_tree.heading(3, text="Name")
        prev_tree.heading(4, text="Number of Statements")
        prev_tree.heading(5, text="Pos:Neu:Neg")
        prev_tree.heading(6, text="Conclusion")
        prev_tree.column(1, width = 8,anchor="c")
        prev_tree.column(2, width = 8,anchor="c")
        prev_tree.column(3, width = 60,anchor="c")
        prev_tree.column(4, width = 30,anchor="c")
        prev_tree.column(5, width = 20,anchor="c")
        prev_tree.column(6, width = 10,anchor="c")
        scroll = ttk.Scrollbar(Bottom_prev_Session, orient="vertical", command=prev_tree.yview)
        scroll.pack(side = LEFT,anchor=NE, fill = 'y')
        
        #prev_tree.bind('<Return>', selectItem)
        #prev_tree.bind('<Double-1>', selectItem)
        prev_tree.configure(yscrollcommand=scroll.set)
        
        s1 = ttk.Style()
        s1.configure('smallBtn.TButton', font=('Helvetica', 14))
        status= Text(Bottom_prev_Session,height=2,width=30,bg='#fff',relief='flat')
        
        send_btn=ttk.Button(Bottom_prev_Session, text="Send Alert",width=10,takefocus=False,style='smallBtn.TButton',command=lambda: send_alert(),state="disabled")
        send_btn.place(x=650,y=390)
                            #********************************************************#
                
                            #********************************************************#                

        PDetails_title=Label(Bottom_P_details,text="Personal Details",font=("Times",18,"underline"),bg='#f1f1b0')
        PDetails_title.place(x=170,y=30,anchor=CENTER)
        
        PDetails_title=Label(Bottom_P_details,text="Gaurdian Details",font=("Times",18,"underline"),bg='#f1f1b0')
        PDetails_title.place(x=630,y=30,anchor=CENTER)
        
        pid= Text(Bottom_P_details,height=2,width=30,bg='#f1f1b0',relief='flat')
        name= Text(Bottom_P_details,height=2,width=60,bg='#f1f1b0',relief='flat') 
        sex= Text(Bottom_P_details,height=2,width=30,bg='#f1f1b0',relief='flat')
        dob= Text(Bottom_P_details,height=2,width=30,bg='#f1f1b0',relief='flat')
        age= Text(Bottom_P_details,height=2,width=30,bg='#f1f1b0',relief='flat') 
        phone= Text(Bottom_P_details,height=2,width=35,bg='#f1f1b0',relief='flat') 
        addr= Text(Bottom_P_details,height=2,width=50,bg='#f1f1b0',relief='flat') 
        g_name= Text(Bottom_P_details,height=2,width=30,bg='#f1f1b0',relief='flat') 
        relation= Text(Bottom_P_details,height=2,width=30,bg='#f1f1b0',relief='flat') 
        gphone= Text(Bottom_P_details,height=2,width=35,bg='#f1f1b0',relief='flat') 
        
        
          
                                    #********************************************************#
        #label=Label(Bottom_Session,text="Individual Details Here",font=("Times",20,"italic"),fg='#718093',bg='#f1f1b0')
        #label.place(x=400,y=30,anchor=CENTER)
        pidS= Text(Bottom_Session,height=2,width=8,bg='#f1f1b0',relief='flat') 
        nameS= Text(Bottom_Session,height=2,width=30,bg='#f1f1b0',relief='flat') 
        sexS= Text(Bottom_Session,height=2,width=20,bg='#f1f1b0',relief='flat') 
        ageS= Text(Bottom_Session,height=2,width=10,bg='#f1f1b0',relief='flat')
        phoneS= Text(Bottom_Session,height=2,width=20,bg='#f1f1b0',relief='flat')
        
        sessionid_label= Text(Bottom_Session,height=2,width=20,bg='#f1f1b0',fg='#944C4D',relief='flat')
        
        inp_label=Label(Bottom_Session,text="Input Statement Here :",font=("",12),bg='#f1f1b0')
        inp_label.place(x=120,y=200,anchor=CENTER)

        statement_entry=Entry(Bottom_Session,width=40,font=("",20),fg='#944C4D',bg='White')
        statement_entry.place(x=400,y=250,anchor=CENTER)
        statement_entry.bind("<Return>",(lambda event: clicked()))
        statement_entry.focus_force()

        waitS_label=Label(Bottom_Session,text="Please Wait! Your sentence is being processed...",font=("Helvetica",10),bg='#f1f1b0')
        progressS = ttk.Progressbar(Bottom_Session, orient = HORIZONTAL, length = 800, mode = 'determinate')

        #-------------------------------------------------------------------------------------------#

root = ThemedTk(theme="scidgreen")
root.geometry("800x750+350+50")
root.title("Sentiment Anlyzer")
root.iconbitmap("emoji.ico")
root.resizable(0,0)

app = Application(master=root)
app.mainloop()


1 record inserted!


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "<ipython-input-11-92ae5ef6ec67>", line 653, in <lambda>
    analyze_btn=ttk.Button(master, text="Analyze",takefocus=False,command=lambda: clicked(),style='my.TButton')
  File "<ipython-input-11-92ae5ef6ec67>", line 487, in clicked
    bar(True)
  File "<ipython-input-11-92ae5ef6ec67>", line 304, in bar
    analyze(statement_entry.get(),session)
  File "<ipython-input-11-92ae5ef6ec67>", line 77, in analyze
    res_stS.configure(text=sentence)
NameError: name 'res_stS' is not defined


In [ ]:
import pandas
from openpyxl import load_workbook
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def analyze(sents):
    for s in sents:
        print(s)
        snt = analyser.polarity_scores(s)
        #print(snt)
        if snt['compound'] >= 0.05 : 
            overall='positive'
            confidence=snt['pos']
        elif snt['compound'] <= - 0.05 : 
            overall='negative'
            confidence=snt['neg']
        else : 
            overall='neutral'
            confidence=snt['neu']
        sentiments.append(overall)
        conf.append(confidence)
    #print(sentiments)
    #print(conf)

    
sentiments=[]
conf=[]
analyser = SentimentIntensityAnalyzer()
     
df = pandas.read_excel('TwitterNeutral.xlsx', sheet_name='Sheet1')
sentences=df['text'].tolist()
analyze(sentences)
wb=load_workbook('TwitterNeutral.xlsx')
ws=wb['Sheet1']
for index,row in list(enumerate(sentiments)):
    cell= 'C%d' %(index+2)
    ws[cell]=row
wb.save('TwitterNeutral.xlsx')


In [6]:
from sklearn.metrics import accuracy_score
import pandas
df1 = pandas.read_excel('TwitterPositive.xlsx', sheet_name='Sheet1')
#df2 = pandas.read_excel('SentimentData.xlsx', sheet_name='Sheet2')
y_pred = df1['pred_sentiment'].tolist()
y_true = df1['sentiment'].tolist()
print(len(y_pred))
print(len(y_true))
print(str(round(accuracy_score(y_true, y_pred)*100,2))+'%')
print(accuracy_score(y_true, y_pred, normalize=False))


1098
1098
88.16%
968


In [9]:
from sklearn.metrics import accuracy_score
import pandas
df1 = pandas.read_excel('TwitterNegative.xlsx', sheet_name='Sheet1')
#df2 = pandas.read_excel('SentimentData.xlsx', sheet_name='Sheet2')
y_pred = df1['pred_sentiment'].tolist()
y_true = df1['sentiment'].tolist()
print(len(y_pred))
print(len(y_true))
print(str(round(accuracy_score(y_true, y_pred)*100,2))+'%')
print(accuracy_score(y_true, y_pred, normalize=False))

1113
1113
64.33%
716


In [8]:
from sklearn.metrics import accuracy_score

import pandas
df1 = pandas.read_excel('TwitterNeutral.xlsx', sheet_name='Sheet1')
#df2 = pandas.read_excel('SentimentData.xlsx', sheet_name='Sheet2')
y_pred = df1['pred_sentiment'].tolist()
y_true = df1['sentiment'].tolist()
print(len(y_pred))
print(len(y_true))
print(str(round(accuracy_score(y_true, y_pred)*100,2))+'%')
print(accuracy_score(y_true, y_pred, normalize=False))

1496
1496
63.37%
948


In [5]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pandas
df1 = pandas.read_excel('Twitter.xlsx', sheet_name='Sheet1')
#df2 = pandas.read_excel('SentimentData.xlsx', sheet_name='Sheet2')
y_pred = df1['pred_sentiment'].tolist()
y_true = df1['sentiment'].tolist()
print(len(y_pred))
print(len(y_true))
print(confusion_matrix(y_true, y_pred))
print(str(round(accuracy_score(y_true, y_pred)*100,2))+'%')
print(accuracy_score(y_true, y_pred, normalize=False))

3707
3707
[[716 154 243]
 [306 948 242]
 [ 46  84 968]]
71.0%
2632
